In [ ]:
"""
Purpose: To compare how the excitatory, inhibitory classifier
compares to that of the manual labels from Stelios


"""

In [ ]:
"""
Ideas on what to do: maybe add a size requirement for axons?



"""

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")

import datajoint_utils as du
from importlib import reload

INFO - 2021-01-26 05:10:56,354 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-01-26 05:10:56,355 - settings - Setting database.user to celiib
INFO - 2021-01-26 05:10:56,356 - settings - Setting database.password to newceliipass
INFO - 2021-01-26 05:10:56,368 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-01-26 05:10:56,369 - settings - Setting enable_python_native_blobs to True
INFO - 2021-01-26 05:10:56,383 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-01-26 05:10:56,690 - settings - Setting enable_python_native_blobs to True


In [3]:
import proofreading_utils as pru
import classification_utils as clu
import neuron_visualizations as nviz

# picking the neuron to look at

In [20]:
minnie,schema = du.configure_minnie_vm()

INFO - 2021-01-26 07:13:36,472 - settings - Setting enable_python_native_blobs to True
INFO - 2021-01-26 07:13:36,769 - settings - Setting enable_python_native_blobs to True


In [18]:
schema.jobs & "table_name='__decomposition_split'" & "status='error'"

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__decomposition_split,01185d62890a925392e301f2235f22f7,error,=BLOB=,Exception: base_node (123) and upstream_node (137) are not connected in the concept network,=BLOB=,celiib@10.28.0.136,at-node9,19,78360,2021-01-26 00:15:25
__decomposition_split,02ec8e9238fa0fc2bb32345caa08c45c,error,=BLOB=,IndexError: index 124610 is out of bounds for axis 0 with size 124610,=BLOB=,celiib@10.28.0.177,at-node50,19,78324,2021-01-25 22:05:54
__decomposition_split,03b2645c30ea2d582e15c9b5442fcebe,error,=BLOB=,IndexError: index 4524 is out of bounds for axis 0 with size 4524,=BLOB=,celiib@10.66.4.64,jr-compute001,20,78130,2021-01-25 21:46:36
__decomposition_split,07ebcb47e106ab01150bdfb2213b23cc,error,=BLOB=,Exception: base_node (44) and upstream_node (47) are not connected in the concept network,=BLOB=,celiib@10.28.0.137,at-node10,19,78329,2021-01-26 00:37:56
__decomposition_split,08242658eb1910118cef2d24440bcd7c,error,=BLOB=,Exception: base_node (22) and upstream_node (21) are not connected in the concept network,=BLOB=,celiib@10.28.0.150,at-node23,19,78353,2021-01-25 23:47:02
__decomposition_split,0bb4bdbc88bbe20f2a82cf87142287d8,error,=BLOB=,IndexError: index 21433 is out of bounds for axis 0 with size 20152,=BLOB=,celiib@10.66.4.27,at-compute005,20,78115,2021-01-25 20:22:43
__decomposition_split,0c1ab060688c9fb5c6fb3c75770a5ead,error,=BLOB=,Exception: base_node (3) and upstream_node (0) are not connected in the concept network,=BLOB=,celiib@10.28.0.156,at-node29,19,78300,2021-01-25 23:21:01
__decomposition_split,134dae03caf9743d364ad93d22f714f5,error,=BLOB=,IndexError: index 206263 is out of bounds for axis 0 with size 52943,=BLOB=,celiib@10.28.0.134,at-node7,19,78327,2021-01-25 22:21:30
__decomposition_split,14036df97d2b1012286fda4feaae83ba,error,=BLOB=,Exception: The number of starting nodes found was not exactly one: [],=BLOB=,celiib@10.28.0.145,at-node18,20,78314,2021-01-25 22:03:11
__decomposition_split,148acf33e87fe66a85d514263df15be4,error,=BLOB=,IndexError: list index out of range,=BLOB=,celiib@10.28.0.147,at-node20,19,78358,2021-01-25 21:56:35


In [39]:
schema.jobs & "table_name='__spine_recalculation'" & "status='error'"

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__spine_recalculation,0043dfa1021df233a927206121267356,error,=BLOB=,"Exception: Segment ID 864691136175024390 should have been in DecompositionSplit but wasnt with n_error_limbs = 2, n_somas = 1",=BLOB=,celiib@10.28.0.83,at-compute004,19,79531,2021-01-26 04:37:36
__spine_recalculation,01eb67595ecadba27d772ba55599c8db,error,=BLOB=,"Exception: Segment ID 864691135994525098 should have been in DecompositionSplit but wasnt with n_error_limbs = 3, n_somas = 2",=BLOB=,celiib@10.28.0.129,at-node2,19,79152,2021-01-26 05:36:07
__spine_recalculation,02ec8e9238fa0fc2bb32345caa08c45c,error,=BLOB=,"Exception: Segment ID 864691135462712516 should have been in DecompositionSplit but wasnt with n_error_limbs = 2, n_somas = 1",=BLOB=,celiib@10.28.0.158,at-node31,21,79141,2021-01-26 06:39:36
__spine_recalculation,033122f2f006a7571b3e0319636e767f,error,=BLOB=,"Exception: Segment ID 864691136116146852 should have been in DecompositionSplit but wasnt with n_error_limbs = 2, n_somas = 1",=BLOB=,celiib@10.28.0.142,at-node15,19,79269,2021-01-26 09:57:51
__spine_recalculation,033bf55873156174ebd5ff962995c65d,error,=BLOB=,"Exception: Segment ID 864691136436406174 should have been in DecompositionSplit but wasnt with n_error_limbs = 2, n_somas = 1",=BLOB=,celiib@10.28.0.152,at-node25,20,79157,2021-01-26 06:53:04
__spine_recalculation,0376e2e8d41a2fc9bbea42e30bcf3cca,error,=BLOB=,"Exception: Segment ID 864691136031741627 should have been in DecompositionSplit but wasnt with n_error_limbs = 2, n_somas = 1",=BLOB=,celiib@10.66.4.64,jr-compute001,19,79253,2021-01-26 09:28:19
__spine_recalculation,043c49068f7695ec9261438ba904dbf3,error,=BLOB=,"Exception: Segment ID 864691136973685276 should have been in DecompositionSplit but wasnt with n_error_limbs = 4, n_somas = 3",=BLOB=,celiib@10.28.0.134,at-node7,19,79104,2021-01-26 09:43:48
__spine_recalculation,046284442be69537d7d0bdd0be867494,error,=BLOB=,"Exception: Segment ID 864691136266437108 should have been in DecompositionSplit but wasnt with n_error_limbs = 2, n_somas = 2",=BLOB=,celiib@10.28.0.134,at-node7,19,79104,2021-01-26 08:23:16
__spine_recalculation,04ae119215968078e26bff9d4db6b011,error,=BLOB=,"Exception: Segment ID 864691136951590367 should have been in DecompositionSplit but wasnt with n_error_limbs = 2, n_somas = 1",=BLOB=,celiib@10.28.0.83,at-compute004,19,79531,2021-01-26 04:37:37
__spine_recalculation,051b9118fbacf01fd2c9154518c7d2c6,error,=BLOB=,"Exception: Segment ID 864691136194008278 should have been in DecompositionSplit but wasnt with n_error_limbs = 3, n_somas = 2",=BLOB=,celiib@10.28.0.157,at-node30,19,79262,2021-01-26 07:20:18


In [45]:
(minnie.Mesh & dict(segment_id=864691136951653599)).delete()

AssertionError: Declaration context is not set

In [ ]:
neuro

In [35]:
len(minnie.DecompositionSplit())

13086

In [ ]:
dj.config["display.limit"] = 50
prediction_and_label_table = minnie.CellTypes() * du.get_exc_inh_classified_table()
new_table = prediction_and_label_table.proj("cell_type_predicted","cell_type")
new_table

# Checking the Splitting

In [ ]:
minnie.DecompositionSplit()

In [ ]:
neuron_obj = (minnie.Decomposition() & dict(segment_id=864691134743737238)).fetch1("decomposition")
neuron_obj

In [ ]:
nviz.plot_soma_limb_concept_network(neuron_obj)

In [ ]:
nviz.visualize_neuron(neuron_obj,
                     limb_branch_dict="all")

In [ ]:
neuron_obj = (minnie.Decomposition() & dict(segment_id=864691134743737238)).fetch1("decomposition")
neuron_obj

# Getting the ids that have different labels

In [ ]:
segment_ids,prediction,labels  = new_table.fetch("segment_id","cell_type_predicted","cell_type")

In [ ]:
error_segments = segment_ids[np.where(prediction != labels)[0]]
error_segments

In [ ]:
minnie.CellTypes()

# checking the statistics of different inhibitory and excitatory neurons

In [ ]:
stats_proj = [
"cell_type_predicted",
"spine_category",
"axon_angle_maximum",
"n_axons",
"n_apicals",
"spine_density",
"n_branches_processed",
"skeletal_length_processed",
"n_branches_in_search_radius",
"skeletal_length_in_search_radius",
]

In [ ]:
import pandas as pd
import pandas_utils as pu

In [ ]:
cell_type_data = minnie.CellTypes.proj(*stats_proj).fetch(as_dict=True)

In [ ]:
cell_df = pu.dicts_to_dataframe(cell_type_data)
non_error_cell = cell_df.query("(not segment_id in @error_segments)",local_dict=dict(error_seegments=error_segments))
non_error_cell

In [ ]:
[non_error_cell][0]

In [ ]:
import numpy_utils as nu
nu.turn_off_scientific_notation()
(non_error_cell[non_error_cell["cell_type_predicted"] == 'inhibitory']["spine_density"].to_numpy())

In [ ]:
pu.plot_histograms_by_grouping(non_error_cell,
                           column_for_grouping="cell_type_predicted",
                           columns=stats_proj,
                           density=True,
                              n_bins=20)

In [ ]:
"""
Conclusion: Changing the spine density will be the best bet

"""

# Trying a decision tree calssification

In [ ]:
import machine_learning_utils as mlu

In [ ]:
col_names = ["spine_density","skeletal_length_processed","axon_angle_maximum","n_axons","n_apicals"]
target_column = "cell_type_predicted"

dec_tree = mlu.decision_tree_sklearn(non_error_cell,
                         target_column=target_column,
                                    feature_columns=col_names)

mlu.plot_decision_tree(dec_tree,
                       col_names)

In [ ]:
mlu.print_tree_structure_description(clf)

# Checking certain neurons

In [ ]:
segment_id = 864691135269778085 #solved by adjusting the parameters
#segment_id = 864691135781479503 #solved
segment_id = 864691135523678660
segment_id = 864691135500208836
segment_id = 864691135937157636 #had to add an extra filter for really unspiny neurons
segment_id = 864691135490141124
segment_id =864691135256136111
segment_id = 864691135730102201
segment_id = 864691135581209284
segment_id = 864691135736483732 #(one that errored on calculate spines)


In [ ]:
# test 2: 
segment_id = 864691135500208836
segment_id = 864691135461969906
segment_id = 864691135730102201
segment_id = 864691135497663251
segment_id = 864691135888524169
segment_id = 864691135488238394
segment_id = 864691136023751737

In [ ]:
du.get_exc_inh_classified_table() & dict(segment_id=segment_id)

In [ ]:
minnie.CellTypes() & dict(segment_id=segment_id)

In [ ]:
minnie.DecompositionSplit() & dict(segment_id=segment_id)

In [ ]:
neuron_obj = (minnie.Decomposition() & dict(segment_id=segment_id)).fetch1("decomposition")
spine_data = (minnie.CellTypes & dict(segment_id=segment_id)).fetch1("spine_data")

In [ ]:
#spine_data = (minnie.CellTypes & dict(segment_id=segment_id)).fetch1("spine_data")
neuron_obj.set_computed_attribute_data(spine_data)

In [ ]:
# len(minnie.Decomposition())
# schema.jobs & "table_name='__decomposition'"   

In [ ]:
neuron_obj.segment_id

In [ ]:
# nviz.visualize_neuron(neuron_obj,
#                        limb_branch_dict="all")

In [ ]:
nviz.plot_spines(neuron_obj)

In [ ]:
neuron_obj.segment_id

In [ ]:
(inh_exc_class,
 spine_category,
 axon_angles,
 n_axons,
 n_apicals,
 neuron_spine_density,
 n_branches_processed,
 skeletal_length_processed,
 n_branches_in_search_radius,
 skeletal_length_in_search_radius
 ) = clu.inhibitory_excitatory_classifier(neuron_obj,
                                    return_spine_classification=True,
                                    return_axon_angles=True,
                                     return_n_axons=True,
                                     return_n_apicals=True,
                                     return_spine_statistics=True)

In [ ]:
(inh_exc_class,
 spine_category,
 axon_angles,
 n_axons,
 n_apicals,
 neuron_spine_density,
 n_branches_processed,
 skeletal_length_processed,
 n_branches_in_search_radius,
 skeletal_length_in_search_radius
 ) 

In [ ]:
clu.apical_classification(neuron_obj,
                         plot_viable_limbs=True,
                         plot_apical=True,
                          apical_width_threshold = 300,
                         verbose=True)

In [ ]:
clu.axon_classification(neuron_obj,
                        plot_axon_like_segments=True,
                        plot_candidates=True,
                       plot_axons=True
                       )

In [ ]:
nviz.visualize_neuron(neuron_obj,
                      limb_branch_dict=dict(L0="all"))

In [ ]:
nviz.visualize_neuron(neuron_obj,
                     limb_branch_dict="all")

# checking a sparsely spiny excitatory

In [ ]:
segment_id = 864691135644605167

In [ ]:
exc_obj = (minnie.Decomposition() & dict(segment_id=segment_id)).fetch1("decomposition")

In [ ]:
exc_obj.calculate_spines()

In [ ]:
nviz.plot_spines(exc_obj)

In [ ]:
(inh_exc_class,
 spine_category,
 axon_angles,
 n_axons,
 n_apicals,
 neuron_spine_density,
 n_branches_processed,
 skeletal_length_processed,
 n_branches_in_search_radius,
 skeletal_length_in_search_radius
 ) = clu.inhibitory_excitatory_classifier(exc_obj,
                                    return_spine_classification=True,
                                    return_axon_angles=True,
                                     return_n_axons=True,
                                     return_n_apicals=True,
                                     return_spine_statistics=True)

In [ ]:
(inh_exc_class,
 spine_category,
 axon_angles,
 n_axons,
 n_apicals,
 neuron_spine_density,
 n_branches_processed,
 skeletal_length_processed,
 n_branches_in_search_radius,
 skeletal_length_in_search_radius
 ) 

In [ ]:
nviz.visualize_neuron(neuron_obj,
                      limb_branch_dict=ns.axon_width_like_segments(neuron_obj,
                           include_ais=True))

In [ ]:
import networkx_utils as xu

In [ ]:
xu.get_coordinate_by_graph_node

In [ ]:
clu.axon_classification(neuron_obj,
                       plot_axons=True)

In [ ]:
ret_col = nviz.visualize_neuron(neuron_obj,
                     limb_branch_dict=dict(L0="all"),
                     return_color_dict=True)

In [ ]:
nviz.plot_limb_concept_network_2D(neuron_obj,
                                 node_colors=ret_col)

In [ ]:
import skeleton_utils as sk
import neuron_searching as ns

In [ ]:
lower_width_bound = 140
upper_width_bound = 500 
spine_threshold = 2
skeletal_distance_threshold = 110000
skeletal_length_threshold = 10000

close_limb_branch_dict = ns.query_neuron(neuron_obj,
                                            functions_list=["skeletal_distance_from_soma_excluding_node","no_spine_median_mesh_center",
                                                            "n_spines","spine_density","skeletal_length"],
                                            query=(
                                                    f"(skeletal_distance_from_soma_excluding_node<{skeletal_distance_threshold})"
                                                   f" and (no_spine_median_mesh_center > {lower_width_bound})"
                                                   f" and (no_spine_median_mesh_center < {upper_width_bound})"
                                                  f" and (n_spines > {spine_threshold})"
                                                f" and skeletal_length > {skeletal_length_threshold} "
                                                  ),)
close_limb_branch_dict

In [ ]:
nviz.visualize_neuron(neuron_obj,
                      visualize_type=["mesh"],
                     limb_branch_dict=close_limb_branch_dict,
                     mesh_color="red",
                     mesh_whole_neuron=True)

In [ ]:
nviz.plot_branches_with_spines(neuron_obj[0][5])

In [ ]:
nviz.plot_spines(neuron_obj)

In [ ]:
nviz.plot_spines(neuron_obj)

In [ ]:
neuron_obj.calculate_spines()

In [ ]:
spine_data = neuron_obj.get_computed_attribute_data(attributes=["spines","spines_volume"])

In [ ]:
import system_utils as su

In [ ]:
su.compressed_pickle(spine_data,"spine_data")

In [ ]:
import neuron_utils as nru
nru.neuron_spine_density(neuron_obj,
                         return_branch_processed_info=True,
                         skeletal_distance_threshold=40000,
                         plot_candidate_branches=True)

In [ ]:
(inh_exc_class,
             spine_category,
             axon_angles,
             n_axons,
             n_apicals,
             neuron_spine_density,
             n_branches_processed,
             skeletal_length_processed,
             n_branches_in_search_radius,
             skeletal_length_in_search_radius
             ) = clu.inhibitory_excitatory_classifier(neuron_obj,
                                                return_spine_classification=True,
                                                return_axon_angles=True,
                                                 return_n_axons=True,
                                                 return_n_apicals=True,
                                                 return_spine_statistics=True)

In [ ]:
(inh_exc_class,
             spine_category,
             axon_angles,
             n_axons,
             n_apicals,
             neuron_spine_density,
             n_branches_processed,
             skeletal_length_processed,
             n_branches_in_search_radius,
             skeletal_length_in_search_radius
             )